# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f44ccb55340>
├── 'a' --> tensor([[ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435]])
└── 'x' --> <FastTreeValue 0x7f44ccb55370>
    └── 'c' --> tensor([[-1.4971,  0.1210, -0.6600,  0.4528],
                        [ 0.7952,  0.5897, -1.6311,  1.3515],
                        [-0.3463,  1.2555,  0.8439,  0.4564]])

In [4]:
t.a

tensor([[ 0.5041,  1.2360,  0.2610],
        [ 0.8510, -0.4521, -0.4435]])

In [5]:
%timeit t.a

58.1 ns ± 0.0882 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f44ccb55340>
├── 'a' --> tensor([[-0.2685, -0.6986,  0.6158],
│                   [-0.5906, -0.9826, -0.8500]])
└── 'x' --> <FastTreeValue 0x7f44ccb55370>
    └── 'c' --> tensor([[-1.4971,  0.1210, -0.6600,  0.4528],
                        [ 0.7952,  0.5897, -1.6311,  1.3515],
                        [-0.3463,  1.2555,  0.8439,  0.4564]])

In [7]:
%timeit t.a = new_value

67.3 ns ± 0.0153 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435]]),
    x: Batch(
           c: tensor([[-1.4971,  0.1210, -0.6600,  0.4528],
                      [ 0.7952,  0.5897, -1.6311,  1.3515],
                      [-0.3463,  1.2555,  0.8439,  0.4564]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5041,  1.2360,  0.2610],
        [ 0.8510, -0.4521, -0.4435]])

In [11]:
%timeit b.a

51.2 ns ± 0.0207 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5049,  0.0287,  0.2918],
               [-0.4945,  0.0478,  0.3922]]),
    x: Batch(
           c: tensor([[-1.4971,  0.1210, -0.6600,  0.4528],
                      [ 0.7952,  0.5897, -1.6311,  1.3515],
                      [-0.3463,  1.2555,  0.8439,  0.4564]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.131 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

831 ns ± 0.45 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 48.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 897 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f44a8119730>
├── 'a' --> tensor([[[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]],
│           
│                   [[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]],
│           
│                   [[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]],
│           
│                   [[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]],
│           
│                   [[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]],
│           
│                   [[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]],
│           
│                   [[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]],
│           
│                   [[ 0.5041,  1.2360,  0.2610],
│                    [ 0.8510, -0.4521, -0.4435]]])
└── 'x' --> <FastTreeValue 0x7f44a833a2e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 71.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f44ccb59ca0>
├── 'a' --> tensor([[ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435],
│                   [ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435],
│                   [ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435],
│                   [ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435],
│                   [ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435],
│                   [ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435],
│                   [ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435],
│                   [ 0.5041,  1.2360,  0.2610],
│                   [ 0.8510, -0.4521, -0.4435]])
└── 'x' --> <FastTreeValue 0x7f44ccb59ac0>
    └── 'c' --> tensor([[-1.4971,  0.1210, -0.6600,  0.4528],
                        [ 0.7952,  0.5897, -1.6311,  1.3515],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 64.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.7 µs ± 833 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]],
       
               [[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]],
       
               [[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]],
       
               [[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]],
       
               [[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]],
       
               [[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]],
       
               [[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]],
       
               [[ 0.5041,  1.2360,  0.2610],
                [ 0.8510, -0.4521, -0.4435]]]),
    x: Batch(
           c: tensor([[[-1.4971,  0.1210, -0.6600,  0.4528],
                       [ 0.7952,  0.5897, -1.6311,  1.3515],
                       [-0.3463,  1.2555,  0.8439,  0.4564]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435],
               [ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435],
               [ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435],
               [ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435],
               [ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435],
               [ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435],
               [ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435],
               [ 0.5041,  1.2360,  0.2610],
               [ 0.8510, -0.4521, -0.4435]]),
    x: Batch(
           c: tensor([[-1.4971,  0.1210, -0.6600,  0.4528],
                      [ 0.7952,  0.5897, -1.6311,  1.3515],
                      [-0.3463,  1.2555,  0.8439,  0.4564],
                      [-1.4971,  0.1210, -0.6600,  0.4528],
                      [ 0.7952,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 225 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
